In [2]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy.linalg as LA
import random
import pandas as pd


In [89]:
df1 = pd.read_csv('data/trajectories-0400-0415.csv')
df2 = pd.read_csv('data/trajectories-0500-0515.csv')
df3 = pd.read_csv('data/trajectories-0515-0530.csv')

## Adding two empty columns for velocities ##
df1['V_X']=np.nan
df1['V_Y']=np.nan

lane=7
filtered_df=df1.loc[df1['Lane_ID']==lane]
list_mergingvehicles=list(filtered_df.Vehicle_ID.unique())
df1_merging_outliers= [5,210,376,388,409,458,465,743,1005,1097,1133,1215,1408,1724,1812,1909,2387,2438,1778,2911]
print(list_mergingvehicles)

[5, 46, 124, 140, 162, 183, 186, 202, 210, 252, 254, 300, 334, 335, 352, 353, 363, 365, 376, 388, 394, 399, 406, 409, 411, 435, 455, 458, 465, 495, 509, 520, 545, 562, 576, 577, 622, 638, 646, 658, 679, 705, 709, 716, 721, 743, 763, 768, 788, 844, 877, 895, 915, 989, 1005, 1020, 1024, 1031, 1064, 1071, 1085, 1094, 1097, 1133, 1137, 1145, 1174, 1177, 1183, 1200, 1215, 1232, 1243, 1267, 1275, 1282, 1290, 1293, 1320, 1323, 1336, 1352, 1368, 1374, 1391, 1396, 1408, 1421, 1498, 1504, 1558, 1568, 1574, 1601, 1617, 1629, 1630, 1674, 1677, 1683, 1703, 1718, 1724, 1733, 1750, 1759, 1762, 1778, 1787, 1802, 1812, 1824, 1831, 1840, 1859, 1867, 1876, 1902, 1909, 1910, 1935, 1940, 1946, 1967, 1969, 1977, 1985, 2000, 2007, 2010, 2027, 2055, 2060, 2064, 2087, 2101, 2102, 2148, 2161, 2178, 2202, 2208, 2239, 2254, 2263, 2280, 2299, 2304, 2314, 2321, 2340, 2349, 2368, 2378, 2387, 2389, 2400, 2410, 2431, 2438, 2457, 2466, 2471, 2758, 2792, 2816, 2817, 2818, 2826, 2836, 2844, 2873, 2877, 2888, 2910, 2911]


In [113]:
df1.iat[0,18]=5
#iat[18]=5
#df1.iloc[0].iat[19]=5

In [118]:
df1.iat[0,5]

48.213

In [ ]:
example7=df1.loc[(df1['Vehicle_ID']==3362) & (df1['Lane_ID']==7)]
example7

In [ ]:
example6=df1.loc[(df1['Vehicle_ID']==list_mergingvehicles[5]) & (df1['Lane_ID']==7)]

In [119]:
####### Filtering data #########
    ####### AND ####### 
#### Calculating Velocities ####
vehicle_row_indices=[]
for vehicle in list_mergingvehicles:
    if vehicle not in df1_merging_outliers:
        #print(vehicle)
        temp=df1.loc[(df1['Vehicle_ID']==vehicle) & (df1['Lane_ID']==7)]
        temp_row_indices=list(temp.loc[temp['Vehicle_ID']==vehicle].index.to_numpy())
               
        temp_extra_time=[]
        extra_time_steps=15
        for i in range(extra_time_steps):
            temp_extra_time.append(temp_row_indices[len(temp_row_indices)-1]+i)
        temp_row_indices.extend(temp_extra_time)
        
        vehicle_row_indices.append([vehicle,temp_row_indices])

        # Velocity #

        for row_index in temp_row_indices[1:]:
                vel_X=0
                vel_Y=0                
                dt=0.1 # Sec
                                
                vel_X=(df1.iat[row_index-1,4]-df1.iat[row_index,4])/dt
                df1.iat[row_index,18]=vel_X
                
                vel_y=(df1.iat[row_index-1,5]-df1.iat[row_index,5])/dt
                df1.iat[row_index,19]=vel_y
                

                


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceding,Following,Space_Headway,Time_Headway
0,1,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
1,1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
2,1,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
3,1,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
4,1,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0


In [ ]:
df1 = pd.read_csv('data/trajectories-0400-0415.csv')
df2 = pd.read_csv('data/trajectories-0500-0515.csv')
df3 = pd.read_csv('data/trajectories-0515-0530.csv')

# Condition on time duration for car following (/100 secs)
duration=1500

# List of all first vehicles that passed the filter
#filtered_vehicles=[]
memory={}
column_index=0  #For the column in memory

for df in [df1,df2,df3] :
    #print(column_index)
    # Fix lane 
    #lane1=1
    #lane2=2
    #filtered_df=df.loc[df['Lane_ID']==any([lane1,lane2]) ]
    lane=1
    filtered_df=df.loc[df['Lane_ID']==lane]

    #print(filtered_df['Lane_ID'])

    filtered_ids=sorted(set(filtered_df["Vehicle_ID"]))
    #filtered_df.reset_index(drop=True)

    # Indicate if a vehicle has been added 
    # to the filtered or discarded
    temp=[0]*len(filtered_ids)      # 0 is not chosen yet # 1 is chosen already # 2 is to discard
    key=iter(filtered_ids)
    val=iter(temp)
    filter_indic=dict(zip(key,val))

    for vehicle in filter_indic:
        count=0
        v_id=vehicle
        #print(filter_indic[v_id])
        if filter_indic[v_id]==0 & count<=500:       
                    
            if len(filtered_df.loc[(filtered_df["Vehicle_ID"]==v_id) & (filtered_df["Following"]!=0) ]["Global_Time"])!=0:
                min_t_follower_exist=min(filtered_df.loc[(filtered_df["Vehicle_ID"]==v_id) & (filtered_df["Following"]!=0) ]["Global_Time"])
                v_follower_id=filtered_df.loc[(filtered_df["Vehicle_ID"]==v_id) & (filtered_df["Global_Time"]==min_t_follower_exist) ]["Following"]
                v_follower_id=v_follower_id.to_numpy()[0]
                            
                if len(filtered_df.loc[(filtered_df["Vehicle_ID"]==v_follower_id) & (filtered_df["Following"]!=0) ]["Global_Time"])!=0:      
                    min_t_follower2_exist=min(filtered_df.loc[(filtered_df["Vehicle_ID"]==v_follower_id) & (filtered_df["Following"]!=0) ]["Global_Time"])
                    v_follower2_id=filtered_df.loc[(filtered_df["Vehicle_ID"]==v_follower_id) & (filtered_df["Global_Time"]==min_t_follower2_exist)]["Following"]
                    v_follower2_id=v_follower2_id.to_numpy()[0]
                    #print(v_id,"v_id")
                    #print(v_follower_id,"v_follower_id")
                    #print(v_follower2_id,"v_follower2_id")
                    # Range of timestamps from which we begin and analyse a interval of 30 secs(duration) to check if they maintain same lane
                    rng_beg=min_t_follower2_exist
                    rng_end=rng_beg+2000

                    for t in range(rng_beg,rng_end,100):
                        
                        t_range=list(range(t,t+duration,100))
                        lane_info_t_range=filtered_df.loc[(filtered_df["Global_Time"]>=t) | (filtered_df["Global_Time"]<=t+duration)]
                                        
                        if (len(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_id)]["Lane_ID"])!=0) & (len(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower_id)]["Lane_ID"])!=0) & (len(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower2_id)]["Lane_ID"])!=0) :
                            
                            lane_info_v=[min(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_id)]["Lane_ID"]),max(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_id)]["Lane_ID"])]
                            lane_info_v_follow=[min(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower_id)]["Lane_ID"]),max(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower_id)]["Lane_ID"])]
                            lane_info_v_follow2=[min(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower2_id)]["Lane_ID"]),max(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower2_id)]["Lane_ID"])]
                            temp=set([lane_info_v[0],lane_info_v[1],lane_info_v_follow[0],lane_info_v_follow[1],lane_info_v_follow2[0],lane_info_v_follow2[1]])

                            if len(temp)==1:
                                v_ids=[v_id,v_follower_id,v_follower2_id]
                                t_init=t
                                t_fin=t+duration
                                timestamps=[t,t+duration]

                                for v in v_ids:
                                    # For a_t-1
                                    v_t_1=df.loc[(df['Vehicle_ID']==v) & (df['Global_Time']==(t_init-100))] 
                                    v_t_2=df.loc[(df['Vehicle_ID']==v) & (df['Global_Time']==(t_init-200))] 
                                    # For a_t
                                    v_t1=df.loc[(df['Vehicle_ID']==v) & (df['Global_Time']==(t_fin+100))]
                                    v_t2=df.loc[(df['Vehicle_ID']==v) & (df['Global_Time']==(t_fin+200))]

                                    # Just changed t_inint for now let us see what happends with t_fin
                                    if v_t_2.size==0:
                                        t_init+=200
                                        t_fin+=200



                                #filtered_vehicles.append([v_ids,timestamps])
                                ## Vechicle_id : Vehicle that follows, Vehicle that follows2 , t_begin(globaltime), t_end(globaltime) ##
                                memory[column_index]=[int(v_id),int(v_follower_id),int(v_follower2_id),int(t_init),int(t_fin)]
                                column_index+=1
                                #print("added",v_ids,timestamps)
                                # Indicate that these vehicles have been added to the filtered data
                                filter_indic[v_id]=filter_indic[v_follower_id]=filter_indic[v_follower2_id]=1
                                count+=1

                                break

  
    print(column_index)

In [5]:
import json

with open('triplets_lane1.json','w') as convert_file:
    convert_file.write(json.dumps(memory))

In [ ]:
df1 = pd.read_csv('data/trajectories-0400-0415.csv')
df2 = pd.read_csv('data/trajectories-0500-0515.csv')
df3 = pd.read_csv('data/trajectories-0515-0530.csv')

# Condition on time duration for car following (/100 secs)
duration=1500

# List of all first vehicles that passed the filter
#filtered_vehicles=[]
memory={}
column_index=0  #For the column in memory
lane_list=[1,2,3]

for lane in lane_list:
    name_is='triplets_lane'+str(lane)+'.json'

    with open(name_is, 'w+') as convert_file:

        for df in [df1,df2,df3] :
            print(column_index)
            # Fix lane 
            #lane1=1
            #lane2=2
            #filtered_df=df.loc[df['Lane_ID']==any([lane1,lane2]) ]
            #lane=1
            filtered_df=df.loc[df['Lane_ID']==lane]
            filtered_ids=sorted(set(filtered_df["Vehicle_ID"]))
            #filtered_df.reset_index(drop=True)

            # Indicate if a vehicle has been added 
            # to the filtered or discarded
            temp=[0]*len(filtered_ids)      # 0 is not chosen yet # 1 is chosen already # 2 is to discard
            key=iter(filtered_ids)
            val=iter(temp)
            filter_indic=dict(zip(key,val))

            for vehicle in filter_indic:
                count=0
                v_id=vehicle
                #print(filter_indic[v_id])
                if filter_indic[v_id]==0 & count<=500:       
                            
                    if len(filtered_df.loc[(filtered_df["Vehicle_ID"]==v_id) & (filtered_df["Following"]!=0) ]["Global_Time"])!=0:
                        min_t_follower_exist=min(filtered_df.loc[(filtered_df["Vehicle_ID"]==v_id) & (filtered_df["Following"]!=0) ]["Global_Time"])
                        v_follower_id=filtered_df.loc[(filtered_df["Vehicle_ID"]==v_id) & (filtered_df["Global_Time"]==min_t_follower_exist) ]["Following"]
                        v_follower_id=v_follower_id.to_numpy()[0]
                                    
                        if len(filtered_df.loc[(filtered_df["Vehicle_ID"]==v_follower_id) & (filtered_df["Following"]!=0) ]["Global_Time"])!=0:      
                            min_t_follower2_exist=min(filtered_df.loc[(filtered_df["Vehicle_ID"]==v_follower_id) & (filtered_df["Following"]!=0) ]["Global_Time"])
                            v_follower2_id=filtered_df.loc[(filtered_df["Vehicle_ID"]==v_follower_id) & (filtered_df["Global_Time"]==min_t_follower2_exist)]["Following"]
                            v_follower2_id=v_follower2_id.to_numpy()[0]
                            #print(v_id,"v_id")
                            #print(v_follower_id,"v_follower_id")
                            #print(v_follower2_id,"v_follower2_id")
                            # Range of timestamps from which we begin and analyse a interval of 30 secs(duration) to check if they maintain same lane
                            rng_beg=min_t_follower2_exist
                            rng_end=rng_beg+2000

                            for t in range(rng_beg,rng_end,100):
                                
                                t_range=list(range(t,t+duration,100))
                                lane_info_t_range=filtered_df.loc[(filtered_df["Global_Time"]>=t) | (filtered_df["Global_Time"]<=t+duration)]
                                                
                                if (len(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_id)]["Lane_ID"])!=0) & (len(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower_id)]["Lane_ID"])!=0) & (len(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower2_id)]["Lane_ID"])!=0) :
                                    
                                    lane_info_v=[min(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_id)]["Lane_ID"]),max(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_id)]["Lane_ID"])]
                                    lane_info_v_follow=[min(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower_id)]["Lane_ID"]),max(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower_id)]["Lane_ID"])]
                                    lane_info_v_follow2=[min(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower2_id)]["Lane_ID"]),max(lane_info_t_range.loc[(lane_info_t_range["Vehicle_ID"]==v_follower2_id)]["Lane_ID"])]
                                    temp=set([lane_info_v[0],lane_info_v[1],lane_info_v_follow[0],lane_info_v_follow[1],lane_info_v_follow2[0],lane_info_v_follow2[1]])

                                    if len(temp)==1:
                                        v_ids=[v_id,v_follower_id,v_follower2_id]
                                        t_init=t
                                        t_fin=t+duration
                                        timestamps=[t,t+duration]

                                        for v in v_ids:
                                            # For a_t-1
                                            v_t_1=df.loc[(df['Vehicle_ID']==v) & (df['Global_Time']==(t_init-100))] 
                                            v_t_2=df.loc[(df['Vehicle_ID']==v) & (df['Global_Time']==(t_init-200))] 
                                            # For a_t
                                            v_t1=df.loc[(df['Vehicle_ID']==v) & (df['Global_Time']==(t_fin+100))]
                                            v_t2=df.loc[(df['Vehicle_ID']==v) & (df['Global_Time']==(t_fin+200))]

                                            # Just changed t_inint for now let us see what happends with t_fin
                                            if v_t_2.size==0:
                                                t_init+=200
                                                t_fin+=200



                                        #filtered_vehicles.append([v_ids,timestamps])
                                        ## Vechicle_id : Vehicle that follows, Vehicle that follows2 , t_begin(globaltime), t_end(globaltime) ##
                                        memory[column_index]=[int(v_id),int(v_follower_id),int(v_follower2_id),int(t_init),int(t_fin)]
                                        column_index+=1
                                        #print("added",v_ids,timestamps)
                                        # Indicate that these vehicles have been added to the filtered data
                                        filter_indic[v_id]=filter_indic[v_follower_id]=filter_indic[v_follower2_id]=1
                                        count+=1

                                        break
            print(column_index)